In [3]:
import pandas as pd
import requests
from datetime import datetime
import time

In [65]:
owner = 'NixOS'  
repo = 'nixpkgs' 

url = f'https://api.github.com/repos/{owner}/{repo}/commits'    #endpoint commits
response = requests.get(url)
response.status_code

200

In [15]:
# get Commitinfo with the GutHub API

TOKEN = 'ghpxxx'   #GitHub API Token

# Repository:
owner = 'NixOS'
repo = 'nixpkgs'
url = f'https://api.github.com/repos/{owner}/{repo}/commits?per_page=100&page=1599'   # Limit at 100 commits per request, start p:1

headers = {
    'Authorization': f'token {TOKEN}',
    'Accept': 'application/vnd.github.v3+json'
}
df53 = pd.DataFrame(columns=['Email', 'Timestamp', 'Company'])  

commit_limit = 5000  # limit 
commit_count = 0

# commit function
def get_commits(url, commit_count, commit_data):
    while url and commit_count < commit_limit:
        response = requests.get(url, headers=headers)
        
        try:
            response.raise_for_status()
            commits = response.json()

            for commit in commits:
                if commit_count >= commit_limit:
                    break
                author_url = commit['author']['url'] if commit['author'] else None
                email = commit['commit']['author']['email']
                timestamp = commit['commit']['author']['date']
                company = None

                # if author_url: try & get company
                if author_url:
                    user_response = requests.get(author_url, headers=headers)
                    if user_response.status_code == 200:
                        user_data = user_response.json()
                        company = user_data.get('company')  

                # add in df
                commit_data.loc[commit_count] = [email, timestamp, company]
                commit_count += 1

            # check next page
            url = response.links.get('next', {}).get('url')

            # waiting time between requests
            time.sleep(1)

        except requests.exceptions.HTTPError as err:
            print(f'HTTP Error: {err}')
            break
    return commit_count, commit_data

final_commit_count, final_commit_data = get_commits(url, 0, df53)
print(f'Total collected commits: {final_commit_count}')
df53

KeyboardInterrupt: 

In [1570]:
df53.duplicated().sum()
#df53= df53.drop_duplicates()

0

In [45]:
#df_new  (df with all the prior commitinfo )

In [ ]:
# anonymize Mails

df_new['Email'] = df_new['Email'].fillna('')
emails = df_new['Email']
anonymized_emails = [f"user{i+1}@anonymized.com" for i in range(len(emails))]
df_new['Email'] = anonymized_emails

In [1556]:
df_new.duplicated().sum()

0

In [1558]:
df_new= pd.concat([df_new, df53], axis=0)         # concatted all DFs gotten from the API requests 
df_new.duplicated().sum()
#df_new= df_new.drop_duplicates()

In [49]:
file_path= '/Users/ironhack/Documents/GitHub/NixOs-Analysis/df_new.csv'  
df_new= pd.read_csv(file_path)

In [21]:
df_new

,Unnamed: 0,Email,Timestamp,Company
0,0,user1@anonymized.com,2024-11-02T22:59:15Z,Inria
1,1,user2@anonymized.com,2024-11-02T22:42:26Z,NaN
2,2,user3@anonymized.com,2024-11-02T22:29:01Z,@lip6
3,3,user4@anonymized.com,2024-11-02T22:24:44Z,PyroNet
4,4,user5@anonymized.com,2024-11-02T22:19:08Z,@lip6
...,...,...,...,...
151898,4881,user151899@anonymized.com,2023-10-10T09:57:13Z,NaN
151899,4886,user151900@anonymized.com,2023-10-10T09:57:12Z,NaN
151900,4890,user151901@anonymized.com,2023-10-10T09:57:11Z,NaN
151901,4894,user151902@anonymized.com,2023-10-10T09:57:10Z,NaN


In [39]:
#df_new= df_new.drop_duplicates()
df_new.duplicated().sum()

In [9]:
df_new

,Unnamed: 0,Email,Timestamp,Company
0,0,user1@anonymized.com,2024-11-02T22:59:15Z,Inria
1,1,user2@anonymized.com,2024-11-02T22:42:26Z,NaN
2,2,user3@anonymized.com,2024-11-02T22:29:01Z,@lip6
3,3,user4@anonymized.com,2024-11-02T22:24:44Z,PyroNet
4,4,user5@anonymized.com,2024-11-02T22:19:08Z,@lip6
...,...,...,...,...
151898,4881,user151899@anonymized.com,2023-10-10T09:57:13Z,NaN
151899,4886,user151900@anonymized.com,2023-10-10T09:57:12Z,NaN
151900,4890,user151901@anonymized.com,2023-10-10T09:57:11Z,NaN
151901,4894,user151902@anonymized.com,2023-10-10T09:57:10Z,NaN


In [19]:
# df_new.to_csv('df_new.csv')    

In [1544]:
# check Remaining requests & limits 

url = "https://api.github.com/rate_limit"
headers = {'Authorization': f'token {TOKEN}'}

response = requests.get(url, headers=headers)
rate_limit_info = response.json()
remaining_requests = rate_limit_info['rate']['remaining']
reset_time = rate_limit_info['rate']['reset']
reset_time_local = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(reset_time))

print(f"Remaining requests: {remaining_requests}")
print(f"Limit reset time (local): {reset_time_local}")

Remaining requests: 4881
Limit reset time (local): 2024-11-10 15:11:16
